# Análise exploratória de dados
### Integrantes:
- Pedro Guilherme
- Luiz Tilio
### Tema: E-commerce
--- 
## Descrição 
Esse trabalho se propõe a realizar uma análise exploratória de dados com base nos conteúdos ministrados durante as aulas de manipulação de planilhas e estatística do Grupo de Estudos de Inteligência Artificial (GEIA). 
## Sobre o dataset
O dataset selecionado contém o registro de dados transacionais durante o período entre 01/12/2010 e 09/12/2011 de uma loja de varejo exclusivamente atuante no modelo online e sediada no Reino Unido. Os principais produtos da loja são presentes para ocasiões diversas. Seus principais clientes são atacadistas.

---

In [1]:
import pandas as pd
import numpy as np

df_cru = pd.read_csv('data.csv', 
            sep= ',',
            encoding= 'ISO-8859-1',
            parse_dates=['InvoiceDate'],
            dtype={
                'InvoiceNo': str,
                'StockCode': str,   
                'CustomerID': str,   
            },
            na_values=['']
            )

df_cru


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680,France


In [2]:
df_cru.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  object        
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(5)
memory usage: 33.1+ MB


## Higienização/Limpeza dos dados

In [3]:
df_sem_nan = df_cru.dropna(axis=0, how='any')
df_sem_duplicado = df_sem_nan.drop_duplicates(keep='first')

df_sem_qtd_neg = df_sem_duplicado[df_sem_duplicado['Quantity'] > 0]
df_sem_preco_neg = df_sem_qtd_neg[df_sem_qtd_neg['UnitPrice'] > 0]

df_limpo = df_sem_preco_neg.reset_index(drop=True)
df_limpo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 392692 entries, 0 to 392691
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    392692 non-null  object        
 1   StockCode    392692 non-null  object        
 2   Description  392692 non-null  object        
 3   Quantity     392692 non-null  int64         
 4   InvoiceDate  392692 non-null  datetime64[ns]
 5   UnitPrice    392692 non-null  float64       
 6   CustomerID   392692 non-null  object        
 7   Country      392692 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(5)
memory usage: 24.0+ MB


In [20]:
#Média de preços
media = round(df_cru['UnitPrice'].mean(), 2)

#Mediana dos preços
mediana = round(df_cru['UnitPrice'].median(), 2)

#Moda dos preços
moda = round(df_cru['UnitPrice'].mode().values[0], 2)

print("Média: ", media)
print("Mediana: ", mediana)
print("Moda: ", moda)

#Variância
variancia = round(df_cru['UnitPrice'].var(), 2)
print("Variância: ", variancia)

#Desvio padrão
desvio_padrao = round(df_cru['UnitPrice'].std(), 2)
print("Desvio padrão: ", desvio_padrao)

#Coeficiente de variação
cv = (desvio_padrao / media) * 100
print("Coeficiente de variação: ", round(cv, 2), "%")

if cv > 30:
    print("Dados heterogêneos")
else:
    print("Dados homogêneos")


#Identificando possiveis outliers
q1 = df_cru['UnitPrice'].quantile(0.25)
q3 = df_cru['UnitPrice'].quantile(0.75)
print("Q1: ", q1)
print("Q3: ", q3)
iqr = q3 - q1
limite_inferior = q1 - 1.5 * iqr
limite_superior = q3 + 1.5 * iqr
possiveis_outliers = df_cru[(df_cru['UnitPrice'] < limite_inferior) | (df_cru['UnitPrice'] > limite_superior)]
possiveis_outliers[['UnitPrice', 'Description']].drop_duplicates(subset='Description').sort_values(by='UnitPrice', ascending=True)


Média:  4.61
Mediana:  2.08
Moda:  1.25
Variância:  9362.47
Desvio padrão:  96.76
Coeficiente de variação:  2098.92 %
Dados heterogêneos
Q1:  1.25
Q3:  4.13


,UnitPrice,Description
1702,8.47,JAM MAKING SET WITH JARS
6854,8.47,CHICK GREY HOT WATER BOTTLE
6902,8.47,ENAMEL WASH BOWL CREAM
6918,8.47,ALARM CLOCK BAKELIKE CHOCOLATE
6919,8.47,ALARM CLOCK BAKELIKE GREEN
...,...,...
4989,295.00,VINTAGE RED KITCHEN CABINET
1814,569.77,DOTCOM POSTAGE
222680,649.50,PICNIC BASKET WICKER 60 PIECES
299982,11062.06,Adjust bad debt
